In [20]:
import yfinance as yf
from datetime import datetime
import pandas as pd

# pd.set_option('display.max_rows', None)  # Show all rows


def calculate_rsi(data, period=14):
    data['Price Change'] = data['Close'].diff()

    data['Gain'] = data['Price Change'].apply(lambda x: x if x > 0 else 0)
    data['Loss'] = data['Price Change'].apply(lambda x: abs(x) if x < 0 else 0)

    data['Avg Gain'] = data['Gain'].rolling(window=period, min_periods=1).mean()
    data['Avg Loss'] = data['Loss'].rolling(window=period, min_periods=1).mean()

    data['RS'] = data['Avg Gain'] / (data['Avg Loss'] + 1e-10)  # div by 0
    data['RSI'] = 100 - (100 / (1 + data['RS']))
    return data

def macd(data, long_period=26, short_period=12, signal_period = 9):
    data['MACD']=data['Close'].ewm(span = short_period, adjust=False).mean() - data['Close'].ewm(span = long_period, adjust=False).mean() # macd = ema12 - ema26

    data['SIG'] = data['MACD'].ewm(span = signal_period, adjust=False).mean()

    data['Histogram'] = data['MACD'] - data['SIG']
    return data

def sma(data, period=20):
    data['SMA'] = data['Close'].rolling(window=period, min_periods=0).mean()
    return data

def ema(data, period=20):
    data['EMA'] = data['Close'].ewm(span = period, adjust=False).mean()
    return data

def bb(data, period=20):
    data['Middle Band'] = data['Close'].rolling(window=period).mean()
    data['Std Dev'] = data['Close'].rolling(window=period).std()  

    data['Upper Band'] = data['Middle Band'] + (2 * data['Std Dev'])
    data['Lower Band'] = data['Middle Band'] - (2 * data['Std Dev'])

    return data.dropna()  # NaN values rmvd

# def strat(data):
#     # Create boolean conditions
#     # data['BUY'] = ((data['RSI'] < 30) & (data['MACD']> data['SIG']) & (data['MACD']< data['Lower Band'])).astype(int) 
#     # data['SELL'] = ((data['RSI'] > 70) & (data['MACD']< data['SIG']) & (data['MACD']> data['Upper Band'])).astype(int)  
#     # return data
#     flag = 0;
#     data["Strat"] = 0
#     data.loc[data["RSI"] < 30, "Strat"] = 1
#     data.loc[data["RSI"] > 70, "Strat"] = 2
#     return data

def strat(data):
    data["Strat"] = 0  # Initialize strategy column
    position = 0  # Track open positions (1 for long, 2 for short)

    for i in range(len(data) - 1):  # Avoid last index error
        if position == 0:  # No open position
            if data["RSI"].iloc[i] < 30:
                data.at[data.index[i], "Strat"] = 1  # Open long
                position = 1
            elif data["RSI"].iloc[i] > 70:
                data.at[data.index[i], "Strat"] = 2  # Open short
                position = 2

        elif position == 1:  # Long position open
            if data["RSI"].iloc[i] > 50:  # RSI recovers → Close long
                data.at[data.index[i], "Strat"] = -1
                position = 0

        elif position == 2:  # Short position open
            if data["RSI"].iloc[i] < 50:  # RSI drops → Close short
                data.at[data.index[i], "Strat"] = -2
                position = 0

    return data


# create a strategy in which when you buy (go long), you open with 1 and close with -1
# create a strategy in which when you sell (go short), you open with 2 and close with -2
# if(1 -> -1 when not fav otherwise 0)
# if(2 -> -2 when not fav otherwise 0)

tickers = ["AAPL", "GOOGL"]
end_time = datetime.now().strftime("%Y-%m-%d")
start_time = "2024-01-01"

for ticker in tickers:
    data = yf.download(ticker, start=start_time, end=end_time)

    data = calculate_rsi(data)
    data = macd(data)
    data = sma(data)
    data = ema(data)
    data = bb(data)

    data['SELL'] = 0
    data['BUY'] = 0
    data['Strat'] = 0

    data = strat(data)

    print(data[['Close', 'RSI', 'MACD', 'SMA', 'EMA', 'Upper Band', 'Lower Band', 'Strat']].tail(5).round(2))
# print(f"Latest RSI: {latest_rsi:.2f}")
# if latest_rsi < 30:
#     print("BUY")
# elif latest_rsi > 70:
#     print("SELL")
# else:
#     print("HOLD")

[*********************100%***********************]  1 of 1 completed


Price        Close    RSI  MACD     SMA     EMA Upper Band Lower Band Strat
Ticker        AAPL                                                         
Date                                                                       
2025-05-13  212.93  60.07 -0.81  203.48  204.49     217.25     189.71     0
2025-05-14  212.33  55.35 -0.12  204.01  205.23     218.30     189.71     0
2025-05-15  211.45  53.09  0.35  204.88  205.83     218.73     191.03     0
2025-05-16  211.26  51.80  0.70  205.60  206.34     219.18     192.03     0
2025-05-19  208.78  47.32  0.77  206.40  206.58     218.64     194.16     0


[*********************100%***********************]  1 of 1 completed

Price        Close    RSI  MACD     SMA     EMA Upper Band Lower Band Strat
Ticker       GOOGL                                                         
Date                                                                       
2025-05-13  159.53  55.33 -0.45  157.26  157.85     166.97     147.56     0
2025-05-14  165.37  57.40  0.19  157.72  158.57     168.06     147.37     0
2025-05-15  163.96  52.51  0.57  158.25  159.08     168.73     147.76     0
2025-05-16  166.19  56.85  1.05  159.00  159.76     169.50     148.50     0
2025-05-19  166.54  57.85  1.43  159.94  160.40     169.51     150.38     0
